In [6]:
import numpy
import copy

In [2]:
from mitiq import cdr

__Sequential Minimal Optimization__ (PRR 2, 043158 (2020)),<br>
(Sequential Minimal Optimization: A Fast Algorithm for Training Support Vector Machines<br>
Microsoft Research Technical Report, 1998, p. 21.)<br>
From the qubit-adapt VQE algorithm we already know the structure of the ansatz:<br>
1. $|\Psi(\mathbf{\theta}^{(n)})\rangle=\exp(\theta^{(n)}_{1}\tau_{1})\exp(\theta^{(n)}_{2}\tau_{2})\ldots \exp(\theta^{(n)}_{k}\tau_{k})|0\rangle$ is a k-parameter ansatz, here $\mathbf{\theta}^{(n)}$ is the k-dim parameter vector after n updates, where $\tau_{j}$ is a multiqubit Pauli string satisfying $\tau_{j}^{2}=I$.<br>
2. Let \begin{align}L^{(n)}(\theta_{j})=\langle 0|U^{(n)\dagger}(\theta_{j})HU^{(n)}(\theta_{j})|0\rangle=a_{1j}^{(n)}\cos(2\theta_{j}-a^{(n)}_{2j})+a_{3j}^{(n)},\end{align} be the associated cost function.<br>
4. Next we compute the cost function at three parameter values $L^{(n)}(\theta^{(n)}_{j})=L_{1}$, $L^{(n)}(\theta^{(n)}_{j}+\frac{\pi}{4})=L_{2}$, $L^{(n)}(\theta^{(n)}_{j}-\frac{\pi}{4})=L_{3}$, from here we can determine $a_{3j}^{(n)}=\frac{L_{2}+L_{3}}{2}$, $a^{(n)}_{2j}=2\theta^{(n)}_{j}-\arctan\left(\frac{L_{3}-L_{2}}{2L_{1}-L_{2}-L_{3}}\right)$, $a^{(n)}_{1j}=(L_{1}-a^{n}_{3j})/\cos\left(\arctan\left(\frac{L_{3}-L_{2}}{2L_{1}-L_{2}-L_{3}}\right)\right)$<br>
5. Finally we compute the theta args corresponding to the extrema of the cost function:
$\theta^{*}_{j}=a^{(n)}_{2j}/2+\pi/2. \text{ if }a^{(n)}_{1j}>0\text{ else } \theta^{*}_{j}=a^{(n)}_{2j}/2$.
6. We repeat step 2-5 for the next multiqubit string in the sequence and after k steps repeat the entire sequence until convergence criteria is met.


In [19]:
#SMO with statevector
def SMO(cost,params,runs=20,tol=1e-4,save_opt_steps=False):
    index=1
    conv_err=1000
    def E_landscape(ind,ang,cost,params):
        params1=copy.deepcopy(params)
        params1[ind]=params1[ind]+ang #ang
        #circ=var_form_base.construct_circuit(parameters=params1)
        E=cost(params1)
        return E.real
    def determine_unknowns(E,cost,params,ind):
        L1=E#_landscape(ind,0,params,Hmat)
        L2=E_landscape(ind,numpy.pi/4.,cost,params)
        L3=E_landscape(ind,-numpy.pi/4.,cost,params)
        ratio=(L3-L2)/(2*L1-L2-L3)
        a3=(L2+L3)/2.
        a2=2*params[ind]-numpy.arctan(ratio)
        a1=(L1-a3)/numpy.cos(numpy.arctan(ratio))
        return a1,a2,a3
    def update(E,cost,params,ind):
        a1,a2,a3=determine_unknowns(E,cost,params,ind)
        thetaStar=a2/2.+numpy.pi/2. if a1>0 else a2/2.
        newParams=copy.deepcopy(params)
        newParams[ind]=thetaStar
        updEnergy=a3-a1 if a1>0 else a3+a1
        return newParams,updEnergy.real
    while conv_err>tol and index<runs:
        print("looped "+str(index)+" times")
        Eold=cost(params)
        init=0
        for i in range(len(params)):#[::-1][0:1]:
            #first run sequential minimial optimization (SMO)  for a given multiqubit operator using 
            #exact analytical form for cost function
            if init==0:
                E=Eold
            ind=i
            params,E=update(E,cost,params,ind)
            if save_opt_steps==True:
                with open('../'+str(U)+'/SMOoptStepsWithSV.txt','a') as f:
                    Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
                    print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
            else:
                continue
            init=init+1  
        conv_err=Eold-E
        print("inner loop error",conv_err)
        index=index+1
    return params,E 

In [20]:
#SMO with qasm
def SMO_qasm(cost,params,runs=20,tol=1e-4,save_opt_steps=False):
    index=1
    conv_err=1000
    data=cost(params)
    t1=time.time()
    def Energy(params):
        backend = Aer.get_backend('statevector_simulator')
        circ=var_form_base.construct_circuit(parameters=params)
        stateVector_0=execute(circ,backend,shots=1024).result().get_statevector()
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(stateVector_0),Hmat),stateVector_0)) 
        return E
    def E_landscape(ind,ang,cost,params):
        params1=copy.deepcopy(params)
        params1[ind]=params1[ind]+ang #ang
        data=cost(params1)
        return data[0],data[1]
    def determine_unknowns(E,MSE1,cost,params,ind):
        L1=E_landscape(ind,0,cost,params)
        L2,MSE2=E_landscape(ind,numpy.pi/4.,cost,params)
        L3,MSE3=E_landscape(ind,-numpy.pi/4.,cost,params)
        ratio=(L3-L2)/(2*L1-L2-L3)
        a3=(L2+L3)/2.
        a2=2*params[ind]-numpy.arctan(ratio)
        a1=(L1-a3)/numpy.cos(numpy.arctan(ratio))
        return a1,a2,a3
    def update(E,MSE,cost,params,ind):
        t1=time.time()
        a1,a2,a3=determine_unknowns(E,MSE,cost,params,ind)
        thetaStar=a2/2.+numpy.pi/2. if a1>0 else a2/2.
        newParams=copy.deepcopy(params)
        newParams[ind]=thetaStar
        data=cost(newParams)
        #print("time taken",time.time()-t1)
        updEnergy,MSE=data[0],data[1]
        return newParams,updEnergy,MSE
    while conv_err>tol and index<runs:
        print("looped "+str(index)+" times")
        data=cost(params)
        Eold=data[0]
        MSE=data[1]
        init=0
        for i in range(len(params)):#[::-1][0:1]:
            #first run sequential minimial optimization (SMO)  for a given multiqubit operator using 
            #exact analytical form for cost function
            if init==0:
                E=Eold
            ind=i
            params,E,MSE=update(E,MSE,cost,params,ind)
            if save_opt_steps==True:
                with open('../'+str(U)+'/SMOoptStepsWithQasm.txt','a') as f:
                    Str=["{:0.16f}".format(params[i]) for i in range(len(params))]
                    Str2=["{:0.16f}".format(elem) for elem in [E,MSE,Energy(params)]]
                    print('['+','.join(Str)+']'+'#'+'['+','.join(Str2)+']',file=f) 
            else:
                continue
            init=init+1  
        conv_err=numpy.abs(Eold-E)
        print("inner loop error",conv_err)
        index=index+1
    return params,E

__Adadelta__(https://d2l.ai/chapter_optimization/adadelta.html, https://ruder.io/optimizing-gradient-descent/)
\begin{align}
\theta_{i+1,l}=\theta_{i,l}-\eta_{i,l}\partial_{\theta_{i,l}}{E}, ~\eta_{i,l}=\sqrt{\frac{r_{i,l}+\epsilon}{v_{i,l}+\epsilon}},
\end{align}<br>
\begin{align}
r_{i,l}=\beta r_{i-1,l}+(1-\beta)(\Delta\theta_{i-1,l}^{2}),
\end{align}<br>
\begin{align}
v_{i,l}= \beta v_{i-1,l}+(1-\beta)\partial_{\theta_{i,l}}{E}^{2}
\end{align}
$\frac{\epsilon}{v_{0,l}+\epsilon}<k\rightarrow \epsilon<\frac{kv_{0,l}}{1-k}$

In [1]:
#Adadelta- statevector
def AdaDelta_sv(cost,params,eps0=1e-8,runs=20,tol=1e-5,save_opt_steps=False):
    v=numpy.zeros((len(params)))*0.0
    m=numpy.zeros((len(params)))*0.0
    delta=numpy.zeros((len(params)))*0.0
    beta=0.9
    rho=0.9
    conv_err=1
    directions= numpy.arange(len(params))
    delta=0
    Max=1e5
    Eold=cost(params).real
    ind=0
    def learningRateEnv(lr0,runs):
        lr_u=lr0+Max*numpy.exp(-(33/runs)*numpy.linspace(0,runs,runs+1))
        lr_d=lr0*(1-numpy.exp(-(33/runs)*numpy.linspace(0,runs,runs+1)))
        return lr_u,lr_d
    def Energy(params):
        backend = Aer.get_backend('statevector_simulator')
        circ=var_form_base.construct_circuit(parameters=params)
        stateVector_0=execute(circ,backend,shots=1024).result().get_statevector()
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(stateVector_0),Hmat),stateVector_0)) 
        return E
    def Grad(cost,params,directions):
        eps=0.005 #
        numpy.pi/4
        shiftedParams1=[numpy.array(params)+numpy.array([0]*ei+[eps]+[0]*(len(params)-ei-1)) for ei in directions]
        shiftedParams2=[numpy.array(params)+numpy.array([0]*ei+[-1*eps]+[0]*(len(params)-ei-1)) for ei in directions]
        Grads=numpy.zeros(len(params))
        for i in tqdm(range(len(directions))):
            estm1=cost(shiftedParams1[i])
            estm2=cost(shiftedParams2[i])
            Grads[directions[i]]=(estm1-estm2)/(2*eps)#numpy.sin(2*eps))
        return Grads
    eps=numpy.array([eps0]*len(params))
    lr0=0.02
    lr_u,lr_d= learningRateEnv(lr0,runs)
    while ((ind<runs) and (conv_err>tol)):
        print(ind,"Index")
        g_sv=Grad(cost,params,directions)
        v = beta*v+(1-beta)*g_sv*g_sv
        lr=numpy.sqrt(delta+eps)/numpy.sqrt(v+eps)
        for i in range(len(lr)):
            if lr[i]>lr_u[ind]:
                eps[i]=lr_u[ind]*lr_u[ind]*v[i]
                lr[i]=lr_u[ind]
                delta[i]=0
            elif lr[i]<lr_d[ind]:
                eps[i]=lr_d[ind]*lr_d[ind]*v[i]
                lr[i]=lr_d[ind]
                delta[i]=0
            else:
                continue
        g=lr*g_sv
        params=params-g
        delta= beta*delta+(1-beta)*g*g
        E=cost(params)
        print("updated energy",E,"learning rate",lr)
        if save_opt_steps==True:
#            with open('../'+str(U)+'/HVAadaDeltaOptStepsWithSV.txt','a') as f:
#                print('['+','.join([str(params[i]) for i in range(len(params))])+']'+'#'+str(E),file=f)
             with open('SMOforHVA.txt','+a') as f:
                Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
                print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
        ind=ind+1 
        conv_err=rho*conv_err+(1-rho)*numpy.abs(Eold-E)
        print("conv err",conv_err)
        Eold=E
    return  params,E   

In [ ]:
def AdaDelta_qasm(cost,params,runs=20,tol=1e-4,num_shots_arr=[2**8,2**9,2**9],save_opt_steps=False):
    string_shots_label=','.join([str(shots) for shots in num_shots_arr])
    v=numpy.zeros((len(params)))*0.0
    delta=numpy.zeros((len(params)))*0.0
    beta=0.9
    rho=0.8
    directions= numpy.arange(len(params))
    delta=0
    conv_err=1
    old_conv_err=1
    Eold=cost(params)
    E_arr=[]
    ind=0
    eps=numpy.array([5e-8]*len(params))#1e-5,5e-6,1e-7,5e-8
    Max=1e5
    lr0=0.015#0.02
    def learningRateEnv(lr0,runs):
        lr_u=lr0+Max*numpy.exp(-(50/runs)*numpy.linspace(0,runs,runs+1))
        lr_d=lr0*(1-numpy.exp(-(50/runs)*numpy.linspace(0,runs,runs+1)))
        return lr_u,lr_d
    def Energy(params):
        backend = Aer.get_backend('statevector_simulator')
        circ=var_form_base.construct_circuit(parameters=params)
        stateVector_0=execute(circ,backend,shots=1024).result().get_statevector()
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(stateVector_0),Hmat),stateVector_0)) 
        return E
    def StochGrad(cost,params,directions,num_shots_arr):
        shiftedParams1=[numpy.array(params)+numpy.array([0]*ei+[numpy.pi/4.]+[0]*(len(params)-ei-1)) for ei in directions]
        shiftedParams2=[numpy.array(params)+numpy.array([0]*ei+[-numpy.pi/4.]+[0]*(len(params)-ei-1)) for ei in directions]
        stochGrads=numpy.zeros((len(params)))
        estm1Arr=map(cost,shiftedParams1)
        estm2Arr=map(cost,shiftedParams2)
        stochGrads=0.5*(numpy.array(list(estm1Arr))-numpy.array(list(estm2Arr)))
        return stochGrads
    lr_u,lr_d= learningRateEnv(lr0,runs)
    flag=0
    flag1=0
    terminate=False
    while ((ind<runs)  and (conv_err>tol)):
        print("index",ind)
        old_conv_err=conv_err
        g_stoch=StochGrad(cost,params,directions,num_shots_arr)
        v = beta*v+(1-beta)*g_stoch*g_stoch
        lr=numpy.sqrt(delta+eps)/numpy.sqrt(v+eps)  #learning rate
        #clipping learning rates, increasing shots and restarting ada delta
        if ((conv_err<1e-4) and (flag==0)):
            flag=1
            num_shots_arr=[2**16,2**16,2**16]
            print("************shots increased**************")
        if ((conv_err<4e-5) and (flag1==0)):
            flag1=1
            num_shots_arr=[2**18,2**20,2**20]
            print("************shots increased**************")    
        for i in range(len(lr)):
            if lr[i]>lr_u[ind]:
                eps[i]=lr_u[ind]*lr_u[ind]*v[i]
                lr[i]=lr_u[ind]
                delta[i]=0
            elif lr[i]<lr_d[ind]:
                eps[i]=lr_d[ind]*lr_d[ind]*v[i]
                lr[i]=lr_d[ind]
                delta[i]=0
            else:
                continue
        g=lr*g_stoch
        params=params-g
        params=params.real
        delta= beta*delta+(1-beta)*g*g
        E=Energy(params)#HamiltonianEstm(params)[0]
        E_arr.append(E)
        print("updated energy",E,"learning rate",lr)#,Energy(params))
        if save_opt_steps==True:
            with open('../'+str(U)+'/adaDeltaOptStepsWithQasm'+string_shots_label+'.txt','a') as f:
                print('['+','.join([str(params[i]) for i in range(len(params))])+']'+'#'+str(E),file=f)
        ind=ind+1 
        conv_err=rho*conv_err+(1-rho)*numpy.abs(Eold-E)
        print("error",conv_err)
        Eold=E
    return  params,E   

27

In [4]:
len([3e-10,3e-10,5e-11,3e-10,3e-10,5e-11,3e-10,5e-11,1e-12,3e-10,3e-10,
                     3e-10,1e-11,3e-10,1e-11,3e-10,3e-10,1e-11,3e-10,3e-10,1e-11,3e-10,
                     3e-10,1e-11,3e-10,3e-10,5e-11,3e-10,3e-10,5e-11,3e-10,5e-11,1e-12,
                     3e-10,3e-10,3e-10,1e-11])

37